In [1]:
# Run it once (in one session)
# !pip install decord
# !pip install einops
# !pip install icecream
# !pip install rarfile
# !pip install unrar

In [2]:
# Imports
import torch
from torch import nn, einsum
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
import torchvision as tv
from torch.utils.data import random_split, DataLoader,Dataset
import time
import random
import math
import decord
import numpy as np
import gc
from einops import rearrange, repeat,reduce
from einops.layers.torch import Rearrange
from PIL import Image
from tqdm.notebook import tqdm
from icecream import ic
from torchvision.datasets import DatasetFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, random_split
import os
import rarfile
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, random_split
import torch.utils.data as data
from torchvision import transforms
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import shutil

import matplotlib.pyplot as plt
%matplotlib inline

from Katna.video import Video
from Katna.writer import KeyFrameDiskWriter

In [3]:
# set device
device ='cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

# DOWNLOADING DATA FROM SOURCE WEBSITE

In [4]:
# # once per session/runtime
# !wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar 

# DATA DIRECTORY SETTINGS

In [5]:
# # once per session/runtime

# rar_path = './hmdb51_org.rar'  
# extract_path = './dataset' 

# with rarfile.RarFile(rar_path, 'r') as rar:
#     rar.extractall(extract_path)

In [6]:
# # once per session/runtime

# direcs = os.listdir(extract_path)

# for i in direcs:
#   with rarfile.RarFile(f"dataset/{i}", 'r') as rar:
#     rar.extractall(f"data/{i.split('.')[0]}")

In [7]:
# once per session/runtime



In [8]:
# once per session/runtime

# Define the path to the dataset


In [9]:
# data_path = './data'  # Path to the original dataset
# dataset_dir = './data_set'  # Destination directory for the modified dataset

# # Iterate over the class folders in the original dataset
# for class_folder in os.listdir(data_path):
#     class_folder_path = os.path.join(data_path, class_folder)

#     # Iterate over the subfolders within each class folder
#     for subfolder in os.listdir(class_folder_path):
#         subfolder_path = os.path.join(class_folder_path, subfolder)

#         # Iterate over the files within each subfolder
#         for file_name in os.listdir(subfolder_path):
#             file_path = os.path.join(subfolder_path, file_name)

#             # Create the destination directory in the modified dataset
#             dest_dir = os.path.join(dataset_dir, subfolder)
#             os.makedirs(dest_dir, exist_ok=True)

#             # Move the file to the destination directory
#             shutil.move(file_path, dest_dir)

# print("Dataset modification complete.") # success message

# DATA LOADING

In [10]:
# Dataset Class
class HMDB51Dataset(data.Dataset):
    def __init__(self, dataset_dir, frames_per_clip=16):
        super().__init__()
        self.dataset_dir = dataset_dir
        self.frames_per_clip = frames_per_clip
        self.video_list = []
        self.labels = []

        # Get the list of video directories
        video_dirs = sorted(os.listdir(dataset_dir))
        
        for label, video_dir in enumerate(video_dirs):
            video_files = os.listdir(os.path.join(dataset_dir, video_dir))
            self.video_list.extend([os.path.join(video_dir, video_file) for video_file in video_files])
            self.labels.extend([label] * len(video_files))

        self.transform = transforms.Compose([
            transforms.Resize((256, 256)),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.video_list)

    def __getitem__(self, idx):
        video_path = os.path.join(self.dataset_dir, self.video_list[idx])
        vid = decord.VideoReader(video_path, ctx=decord.cpu(0))
        nframes = len(vid)

        # If the number of frames in the video is less than frames_per_clip, repeat the frames
        if nframes <= self.frames_per_clip:
            frame_idxs = torch.arange(0, self.frames_per_clip) % nframes
        # Else, sample uniformly separated frames
        else:
            frame_idxs = torch.linspace(0, nframes - 1, self.frames_per_clip).long()

        frames = []
        for frame_idx in frame_idxs:
            frame_idx = frame_idx.item()  # Convert to scalar value
            frame = Image.fromarray(vid[frame_idx].asnumpy())
            frame = self.transform(frame)
            frames.append(frame)

        frames = torch.stack(frames)

        label = self.labels[idx]
        return frames, label

In [11]:
# Directory of the HMDB51 dataset
dataset_dir = "./data"
frames_per_clip = 8
# Instantiate the dataset
hmdb51_dataset = HMDB51Dataset(dataset_dir, frames_per_clip=frames_per_clip)

# TRAIN TEST SPLIT

In [12]:
# Split the dataset into train, validation, and test sets
train_len = int(0.7 * len(hmdb51_dataset))
test_len = len(hmdb51_dataset) - train_len - (len(hmdb51_dataset) - train_len)//2
train_data, val_data,test_data = torch.utils.data.random_split(hmdb51_dataset, [train_len,test_len, len(hmdb51_dataset) - test_len - train_len])

# Data loading parameters
batch_size = 32
test_batch_size = 1
num_workers = 0
pin_memory = True
num_classes = len(set(hmdb51_dataset.labels))



In [13]:
# Dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers, pin_memory=pin_memory)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=test_batch_size, num_workers=num_workers, pin_memory=pin_memory)

In [14]:
# Instantiate and create train-val-test split
# train_val_data = HMDB51Dataset(dataset_dir, frames_per_clip=frames_per_clip)
# train_len = int(0.7 * len(hmdb51_dataset))
# test_len = len(hmdb51_dataset) - train_len - (len(hmdb51_dataset) - train_len)//2
# train_val_split = [train_len,test_len, len(hmdb51_dataset) - test_len - train_len]
# train_data, val_data,test_data = random_split(train_val_data, train_val_split)
# test_data = HMDB51Dataset(dataset_dir, frames_per_clip=frames_per_clip)

# Print the number of samples in each split
print(f"Train samples: {len(train_data)}")
print(f"Validation samples: {len(val_data)}")
print(f"Test samples: {len(test_data)}")

Train samples: 4736
Validation samples: 1015
Test samples: 1015


# DEFINING MODEL

In [15]:
class MLP(nn.Module):
    """
    Builds a simple feed forward network
    Args:
    - dim: (int) - inner dimension of embeddings
    - inner_dim: (int) - dimension of transformer head
    - n_class: (int) - number of output classes
    - encoder: the DinoVisionTransformer encoder
    """
    def __init__(self, dim, inner_dim, n_class, encoder):
        super().__init__()
        self.encoder = encoder
        self.mlp = nn.Sequential(
            nn.Linear(dim,inner_dim),
            nn.Linear(inner_dim,inner_dim/2),
            nn.Linear(inner_dim/2, n_class)
        )

    def forward(self, x):
        B, T, C, H, W = x.shape
        x = x.reshape(B*T, C, H, W)
        output = self.encoder(x)
        output = output.reshape(B, T, -1)
        avg = output.mean(dim=1)  # Average pooling over time
        return self.mlp(avg)

In [16]:
# Instantiate the model
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dinov2_vits14.to(device)
for param in dinov2_vits14.parameters():
    param.requires_grad = False
for i in range(12):
    #dinov2_vits14.blocks[i].add_module('ln',nn.LayerNorm(384))
    dinov2_vits14.blocks[i].add_module('fc1',nn.Linear(384, 384, bias=True))
    dinov2_vits14.blocks[i].add_module('conv',nn.Conv3d(384, 384, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), groups=384))
    dinov2_vits14.blocks[i].add_module('fc2',nn.Linear(384, 384, bias=True))
model = MLP(384, 512, 51, dinov2_vits14)
model.to(device)

Using cache found in /home/z3qian/.cache/torch/hub/facebookresearch_dinov2_main


MLP(
  (encoder): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (

## SPECIFYING HYPER-PARAMETERS

In [17]:
# Define the loss function and optimizer
lr=0.01
epochs = 15
decay_rate = 0.95
loss_criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=lr,momentum=0.9,weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=decay_rate)  # stepwise learning rate decay


# MODEL TRAINING & EVALUATION

In [18]:
# Training loop
val_best = 0
for epoch in range(1, epochs + 1):
    model.train()
    total_epoch_loss = 0
    for batch_id, (video_data, labels) in tqdm(enumerate(train_loader)):
        video_data, labels = video_data.to(device), labels.to(device)
        optimizer.zero_grad()
        prediction = model(video_data)
        loss = loss_criterion(prediction, labels)
        loss.backward()
        optimizer.step()
        total_epoch_loss += loss.item()
        
        corrects = (torch.argmax(prediction,dim=1)==labels).sum()
        bacc = corrects/batch_size
        del video_data
        del labels

        gc.collect()
        print(f"\n[Train Epoch]: {epoch} Train Loss: {loss.item()}, Batch Acc is {bacc.item()}")

        # Add any additional training metrics/logging you need
    scheduler.step()
    # Perform validation at the end of each epoch
    model.eval()
    total_loss = 0
    corrects = 0
    with torch.no_grad():
        for batch_id, (video_data, labels) in enumerate(val_loader):
            video_data, labels = video_data.to(device), labels.to(device)
            prediction = model(video_data)
            loss = loss_criterion(prediction, labels)
            total_loss += loss.item()
            corrects += (torch.argmax(prediction, dim=1) == labels).sum()
            del video_data
            del labels

            gc.collect()
    accuracy = corrects / (len(val_loader) * batch_size)
    print(f"\n[Val Epoch]: {epoch} , Accuracy: {accuracy}, Valid Loss: {total_loss / len(val_loader)}")
    if accuracy > val_best:
        torch.save(model,'best_hmdb_model.pth')
        val_best = accuracy

0it [00:00, ?it/s]


[Train Epoch]: 1 Train Loss: 4.284613132476807, Batch Acc is 0.03125

[Train Epoch]: 1 Train Loss: 4.129614353179932, Batch Acc is 0.0625

[Train Epoch]: 1 Train Loss: 3.8956010341644287, Batch Acc is 0.125

[Train Epoch]: 1 Train Loss: 3.9118852615356445, Batch Acc is 0.15625

[Train Epoch]: 1 Train Loss: 4.209933280944824, Batch Acc is 0.0

[Train Epoch]: 1 Train Loss: 3.4511027336120605, Batch Acc is 0.125

[Train Epoch]: 1 Train Loss: 3.4507508277893066, Batch Acc is 0.1875

[Train Epoch]: 1 Train Loss: 3.2362844944000244, Batch Acc is 0.1875

[Train Epoch]: 1 Train Loss: 3.5313937664031982, Batch Acc is 0.15625

[Train Epoch]: 1 Train Loss: 3.0278358459472656, Batch Acc is 0.34375

[Train Epoch]: 1 Train Loss: 2.563516855239868, Batch Acc is 0.21875

[Train Epoch]: 1 Train Loss: 2.864976167678833, Batch Acc is 0.28125

[Train Epoch]: 1 Train Loss: 2.4186110496520996, Batch Acc is 0.375

[Train Epoch]: 1 Train Loss: 2.5495197772979736, Batch Acc is 0.28125

[Train Epoch]: 1 Train 

0it [00:00, ?it/s]


[Train Epoch]: 2 Train Loss: 0.9805267453193665, Batch Acc is 0.75

[Train Epoch]: 2 Train Loss: 0.7623410820960999, Batch Acc is 0.8125

[Train Epoch]: 2 Train Loss: 0.8012363910675049, Batch Acc is 0.71875

[Train Epoch]: 2 Train Loss: 1.3614611625671387, Batch Acc is 0.59375

[Train Epoch]: 2 Train Loss: 0.8464748859405518, Batch Acc is 0.75

[Train Epoch]: 2 Train Loss: 0.7916880249977112, Batch Acc is 0.6875

[Train Epoch]: 2 Train Loss: 1.2786709070205688, Batch Acc is 0.625

[Train Epoch]: 2 Train Loss: 0.6058558225631714, Batch Acc is 0.84375

[Train Epoch]: 2 Train Loss: 0.7307132482528687, Batch Acc is 0.75

[Train Epoch]: 2 Train Loss: 0.7183325290679932, Batch Acc is 0.84375

[Train Epoch]: 2 Train Loss: 0.8722740411758423, Batch Acc is 0.71875

[Train Epoch]: 2 Train Loss: 0.688554584980011, Batch Acc is 0.78125

[Train Epoch]: 2 Train Loss: 0.6867672801017761, Batch Acc is 0.78125

[Train Epoch]: 2 Train Loss: 0.7314951419830322, Batch Acc is 0.75

[Train Epoch]: 2 Train

0it [00:00, ?it/s]

...


[Train Epoch]: 13 Train Loss: 0.40730953216552734, Batch Acc is 0.96875

[Train Epoch]: 13 Train Loss: 0.25386330485343933, Batch Acc is 0.96875

[Train Epoch]: 13 Train Loss: 0.2697989046573639, Batch Acc is 0.96875

[Train Epoch]: 13 Train Loss: 0.42297694087028503, Batch Acc is 0.875

[Train Epoch]: 13 Train Loss: 0.41746342182159424, Batch Acc is 0.8125

[Train Epoch]: 13 Train Loss: 0.24360665678977966, Batch Acc is 0.9375

[Train Epoch]: 13 Train Loss: 0.31062188744544983, Batch Acc is 0.9375

[Train Epoch]: 13 Train Loss: 0.48337557911872864, Batch Acc is 0.8125

[Train Epoch]: 13 Train Loss: 0.32167521119117737, Batch Acc is 0.90625

[Train Epoch]: 13 Train Loss: 0.1774710714817047, Batch Acc is 1.0

[Train Epoch]: 13 Train Loss: 0.27419137954711914, Batch Acc is 0.96875

[Train Epoch]: 13 Train Loss: 0.32844603061676025, Batch Acc is 0.90625

[Train Epoch]: 13 Train Loss: 0.2593860328197479, Batch Acc is 0.9375

[Train Epoch]: 13 Train Loss: 0.333560973405838, Batch Acc is 0.

0it [00:00, ?it/s]


[Train Epoch]: 14 Train Loss: 0.3262074887752533, Batch Acc is 0.9375

[Train Epoch]: 14 Train Loss: 0.4186933934688568, Batch Acc is 0.875

[Train Epoch]: 14 Train Loss: 0.30461978912353516, Batch Acc is 0.9375

[Train Epoch]: 14 Train Loss: 0.252218097448349, Batch Acc is 0.9375

[Train Epoch]: 14 Train Loss: 0.3092147409915924, Batch Acc is 0.90625

[Train Epoch]: 14 Train Loss: 0.2434440404176712, Batch Acc is 0.96875

[Train Epoch]: 14 Train Loss: 0.21588754653930664, Batch Acc is 0.9375

[Train Epoch]: 14 Train Loss: 0.4044225215911865, Batch Acc is 0.9375

[Train Epoch]: 14 Train Loss: 0.2894948124885559, Batch Acc is 0.96875

[Train Epoch]: 14 Train Loss: 0.19970430433750153, Batch Acc is 1.0

[Train Epoch]: 14 Train Loss: 0.23009400069713593, Batch Acc is 0.9375

[Train Epoch]: 14 Train Loss: 0.379364013671875, Batch Acc is 0.90625

[Train Epoch]: 14 Train Loss: 0.41574785113334656, Batch Acc is 0.84375

[Train Epoch]: 14 Train Loss: 0.38392600417137146, Batch Acc is 0.875

[

0it [00:00, ?it/s]


[Train Epoch]: 15 Train Loss: 0.42589235305786133, Batch Acc is 0.875

[Train Epoch]: 15 Train Loss: 0.3013838231563568, Batch Acc is 0.90625

[Train Epoch]: 15 Train Loss: 0.28396695852279663, Batch Acc is 0.96875

[Train Epoch]: 15 Train Loss: 0.3116600215435028, Batch Acc is 0.90625

[Train Epoch]: 15 Train Loss: 0.31639227271080017, Batch Acc is 0.90625

[Train Epoch]: 15 Train Loss: 0.3083488941192627, Batch Acc is 0.9375

[Train Epoch]: 15 Train Loss: 0.3796517848968506, Batch Acc is 0.90625

[Train Epoch]: 15 Train Loss: 0.35170358419418335, Batch Acc is 0.84375

[Train Epoch]: 15 Train Loss: 0.43576958775520325, Batch Acc is 0.875

[Train Epoch]: 15 Train Loss: 0.3819088637828827, Batch Acc is 0.875

[Train Epoch]: 15 Train Loss: 0.22537745535373688, Batch Acc is 0.9375

[Train Epoch]: 15 Train Loss: 0.24649865925312042, Batch Acc is 0.96875

[Train Epoch]: 15 Train Loss: 0.3536668121814728, Batch Acc is 0.90625

[Train Epoch]: 15 Train Loss: 0.3075004816055298, Batch Acc is 1

In [19]:
# Save the trained model
#torch.save(model, "hmdb_st_model.pth")
model = torch.load('best_hmdb_model.pth')
model.to(device)

MLP(
  (encoder): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
        (

In [20]:
def test_model(loader):

    model.eval()
    corrects=0
    total_loss = 0
    with torch.no_grad():
        for batch_id, (input_data,labels) in enumerate(loader):
            
            input_data = input_data.to(device)
            
            labels = labels.to(device)
            prediction = model(input_data)
            loss = loss_criterion(prediction,labels)
            total_loss += loss.item()
            corrects+= (torch.argmax(prediction,dim=1)==labels).sum()
    
    accuracy = corrects/(len(loader)*test_batch_size)
    print(f"Test Accuracy: {accuracy}, Test Loss: {total_loss}")

    return accuracy

In [21]:
test_model(test_loader)

Test Accuracy: 0.6896551847457886, Test Loss: 1142.5351494767528


tensor(0.6897, device='cuda:0')